In [1]:
# impirt important packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
# read data
airline_data = pd.read_csv('../../data/dataset/airline.csv')
# cache_map_data = pd.read_csv('../../data/dataset/cache_map.csv')
group_data = pd.read_csv('../../data/dataset/group.csv')
order_data = pd.read_csv('../../data/dataset/order.csv')
day_schedule_data = pd.read_csv('../../data/dataset/day_schedule.csv')

#
column_names = ['AirportID', 'Name', 'City', 'Country', 'IATA', 'ICAO', 'Latitude', 'Longitude', 
              'Altitude', 'Timezone', 'DST', 'TZ', 'Type','Source']
df_airport = pd.read_csv('../../data/airport.csv', names=column_names) #ref: https://openflights.org/data.html#airport

# test and train data
train_data = pd.read_csv('../../data/training-set.csv')
test_data = pd.read_csv('../../data/testing-set.csv')

/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


### 處理訂單變數

In [3]:
order_data['order_date_dt'] = pd.to_datetime(order_data.order_date, format='%d-%b-%y') # ref: http://strftime.org/
order_data['order_year'] = order_data.order_date_dt.dt.year
order_data['order_month'] = order_data.order_date_dt.dt.month
order_data['order_day'] = order_data.order_date_dt.dt.day
order_data['order_dayofyear'] = order_data.order_date_dt.dt.dayofyear
order_data['order_weekday'] = order_data.order_date_dt.dt.dayofweek # 0=Mon, 6=Sun
order_data['unit']=order_data.unit.str.replace('unit_value_', '').astype(int)
order_data['source_1']=order_data.source_1.str.replace('src1_value_', '')
order_data['source_2']=order_data.source_2.str.replace('src2_value_', '')

### 處理group變數

In [4]:
group_data['sub_line'] = group_data.sub_line.str.replace('subline_value_','')
group_data['area'] = group_data.area.str.replace('area_value_','')
group_data['begin_date_dt'] = pd.to_datetime(group_data.begin_date, format='%d-%b-%y')
group_data['begin_year'] = group_data.begin_date_dt.dt.year
group_data['begin_month'] = group_data.begin_date_dt.dt.month
group_data['begin_day'] = group_data.begin_date_dt.dt.day
group_data['begin_dayofyear'] = group_data.begin_date_dt.dt.dayofyear
group_data['begin_weekday'] = group_data.begin_date_dt.dt.dayofweek # 0=Mon, 6=Sun
group_data['end_date_dt'] = group_data.begin_date_dt + group_data['days'].apply(np.ceil).apply(lambda x: pd.Timedelta(x, unit='D'))
group_data['end_year'] = group_data.end_date_dt.dt.year
group_data['end_month'] = group_data.end_date_dt.dt.month
group_data['end_day'] = group_data.end_date_dt.dt.day
group_data['end_dayofyear'] = group_data.end_date_dt.dt.dayofyear
group_data['end_weekday'] = group_data.end_date_dt.dt.dayofweek # 0=Mon, 6=Sun

### 處理航班變數

In [5]:
#
airline_data['dst_airport'] = airline_data.dst_airport.str.replace('([A-Z]+) ([^A-Z]+)','\\1')
airline_data['src_airport'] = airline_data.src_airport.str.replace('([A-Z]+) ([^A-Z]+)','\\1')
airline_data['go_back'] = airline_data.go_back.str.replace('去程', 'go')
airline_data['go_back'] = airline_data.go_back.str.replace('回程', 'back')

# unify time, compute the travel time for each routes
airline_data['src_IATA'] = airline_data.src_airport.str.replace('([A-Z]+) ([^A-Z])+', '\\1')
airline_data['dst_IATA'] = airline_data.dst_airport.str.replace('([A-Z]+) ([^A-Z])+', '\\1')
tmp = pd.merge(airline_data, df_airport[['IATA','TZ']], left_on='src_IATA', right_on='IATA', how='left')
airline_data = tmp.rename(index=str, columns={'TZ':'src_TZ'}).drop(['IATA'], axis=1)
tmp = pd.merge(airline_data, df_airport[['IATA','TZ']], left_on='dst_IATA', right_on='IATA', how='left')
airline_data = tmp.rename(index=str, columns={'TZ':'dst_TZ'}).drop(['IATA'], axis=1)
airline_data['fly_time'] = pd.to_datetime(airline_data.fly_time, format='%Y/%m/%d %H:%M')
airline_data['arrive_time'] = pd.to_datetime(airline_data.arrive_time, format='%Y/%m/%d %H:%M')

def src_time_utc(x):
    try: 
        res = x['fly_time'].tz_localize(x['src_TZ']).tz_convert('UTC')
    except:
        res = None # 因為有些機場沒有抓到對應time-zone (可後續用手動補)
    return res

def dst_time_utc(x):
    try:
        res = x['arrive_time'].tz_localize(x['dst_TZ']).tz_convert('UTC')
    except:
        res = None
    return res

airline_data['arrive_time_utc'] = airline_data.apply(dst_time_utc, axis=1)
airline_data['fly_time_utc'] = airline_data.apply(src_time_utc, axis=1)
airline_data['traval_time'] = airline_data['arrive_time_utc'] - airline_data['fly_time_utc']

# derive number of routes for each group
go_route_num=airline_data[airline_data['go_back']=='go'].groupby('group_id')['dst_airport'].nunique()
back_route_num=airline_data[airline_data['go_back']=='back'].groupby('group_id')['dst_airport'].nunique()
df_route_num = pd.merge(go_route_num.to_frame('go_route_num'), back_route_num.to_frame('back_route_num'), left_index=True, right_index=True,how='outer')

# derive total travel time for each group
def mean(x):
    return x.sum()/x.count()

#
tmp=airline_data[airline_data.go_back=='go'].groupby(['group_id','dst_IATA'])['traval_time'].apply(mean)
#print(tmp.head())
go_time=tmp.to_frame().groupby(['group_id'])['traval_time'].sum()

#
tmp=airline_data[airline_data.go_back=='back'].groupby(['group_id','dst_IATA'])['traval_time'].apply(mean)
#print(tmp.head())
back_time=tmp.to_frame().groupby(['group_id'])['traval_time'].sum()

#
df_total_travel_time = pd.merge(go_time.to_frame('go_time'), back_time.to_frame('back_time'), left_index=True, right_index=True,how='outer')

In [6]:
# merge following two df with order_id on group_id 
print(df_total_travel_time.head())
print(df_route_num.head())

                                  go_time back_time
group_id                                           
00018d44e062e3db9dc0460a5c78f0ee 04:25:00  04:50:00
000858b64a52e724cd008bbfe2fc8cdd 02:15:00  02:35:00
000a91291573c4b8b44300de8136b86b 03:15:00  03:00:00
000d1cccbb593a8210840a43457aa5e2 02:25:00  02:20:00
0012d75d4bfe483a414b09b50b548f58 02:30:00  03:00:00
                                  go_route_num  back_route_num
group_id                                                      
00018d44e062e3db9dc0460a5c78f0ee           1.0             1.0
000858b64a52e724cd008bbfe2fc8cdd           1.0             1.0
000a91291573c4b8b44300de8136b86b           1.0             1.0
000d1cccbb593a8210840a43457aa5e2           1.0             1.0
0012d75d4bfe483a414b09b50b548f58           1.0             1.0


In [7]:
df_route_num.describe()

,go_route_num,back_route_num
count,48795.000000,48755.000000
mean,1.069905,1.067501
std,0.275391,0.268732
min,1.000000,1.000000
25%,1.000000,1.000000
50%,1.000000,1.000000
75%,1.000000,1.000000
max,5.000000,4.000000


In [8]:
df_total_travel_time.describe() 

,go_time,back_time
count,48795,48755
mean,0 days 05:11:58.263756,0 days 05:02:34.065429
std,1 days 15:56:56.208220,0 days 04:03:28.856158
min,-1 days +08:55:00,-1 days +02:40:00
25%,0 days 02:30:00,0 days 02:40:00
50%,0 days 03:10:00,0 days 03:20:00
75%,0 days 04:35:00,0 days 04:50:00
max,365 days 17:40:00,1 days 15:25:00


In [13]:
df_total_travel_time = df_total_travel_time.reset_index()
df_total_travel_time['group_id'] = df_total_travel_time.group_id.apply(str)
df_route_num = df_route_num.reset_index()
df_route_num['group_id'] = df_route_num.group_id.apply(str)
tmp = airline_data[airline_data['go_back']=='go'].sort_values(by='fly_time')
df_airline_fly_time = tmp.groupby('group_id').first().reset_index()[['group_id', 'fly_time']]
df_airline_fly_time['group_id'] = df_airline_fly_time.group_id.apply(str)

> need to fix the cross year trip

In [17]:
tmp = pd.merge(order_data, df_total_travel_time, on='group_id', how='left')
tmp = pd.merge(tmp, df_route_num, on='group_id', how='left')
tmp = pd.merge(tmp, df_airline_fly_time, on='group_id', how='left')
df_predictors = pd.merge(tmp, group_data.drop(['product_name','promotion_prog'], axis=1), on='group_id', how='left').drop('order_date', axis=1)
df_predictors['date_diff_order_begin'] = df_predictors.fly_time - df_predictors.order_date_dt  

In [19]:
df_predictors.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 396915 entries, 0 to 396914
Data columns (total 35 columns):
order_id                 396915 non-null object
group_id                 396915 non-null object
source_1                 396915 non-null object
source_2                 396915 non-null object
unit                     396915 non-null int64
people_amount            396915 non-null int64
order_date_dt            396915 non-null datetime64[ns]
order_year               396915 non-null int64
order_month              396915 non-null int64
order_day                396915 non-null int64
order_dayofyear          396915 non-null int64
order_weekday            396915 non-null int64
go_time                  134507 non-null timedelta64[ns]
back_time                134523 non-null timedelta64[ns]
go_route_num             134507 non-null float64
back_route_num           134523 non-null float64
fly_time                 134507 non-null datetime64[ns]
sub_line                 134771 non-null obj

In [21]:
df_predictors.describe()

,unit,people_amount,order_year,order_month,order_day,order_dayofyear,order_weekday,go_time,back_time,go_route_num,...,begin_month,begin_day,begin_dayofyear,begin_weekday,end_year,end_month,end_day,end_dayofyear,end_weekday,date_diff_order_begin
count,396915.000000,396915.000000,396915.000000,396915.000000,396915.000000,396915.000000,396915.000000,134507,134523,134507.000000,...,134771.000000,134771.000000,134771.000000,134771.000000,134771.000000,134771.000000,134771.000000,134771.000000,134771.000000,134507
mean,31.586612,3.207004,2017.075618,6.321285,15.460083,176.584095,2.384233,0 days 08:05:42.672054,0 days 06:39:02.033183,1.168958,...,5.181181,16.095384,142.429744,3.153460,2017.749494,5.339324,15.332460,146.460997,2.864207,59 days 18:41:41.283204
std,30.108192,3.147724,0.535983,3.445579,8.666361,105.970022,1.779056,4 days 02:44:02.412028,0 days 04:36:57.274862,0.417474,...,2.991993,8.910586,90.479228,1.920811,0.433306,2.933142,8.950419,89.315407,2.098590,41 days 19:29:15.191395
min,1.000000,1.000000,2016.000000,1.000000,1.000000,1.000000,0.000000,-1 days +13:40:00,-1 days +20:30:00,1.000000,...,1.000000,1.000000,1.000000,0.000000,2017.000000,1.000000,1.000000,1.000000,0.000000,-312 days +16:35:00
25%,6.000000,2.000000,2017.000000,3.000000,8.000000,85.000000,1.000000,0 days 02:45:00,0 days 02:55:00,1.000000,...,3.000000,8.000000,63.000000,2.000000,2017.000000,3.000000,7.000000,68.000000,1.000000,28 days 09:15:00
50%,19.000000,2.000000,2017.000000,6.000000,15.000000,170.000000,2.000000,0 days 03:50:00,0 days 04:10:00,1.000000,...,5.000000,17.000000,129.000000,3.000000,2018.000000,5.000000,15.000000,132.000000,3.000000,50 days 18:20:00
75%,53.000000,4.000000,2017.000000,9.000000,23.000000,270.000000,4.000000,0 days 12:55:00,0 days 11:45:00,1.000000,...,8.000000,24.000000,217.000000,5.000000,2018.000000,8.000000,23.000000,220.000000,5.000000,83 days 14:35:00
max,131.000000,50.000000,2018.000000,12.000000,31.000000,366.000000,6.000000,365 days 17:40:00,1 days 15:25:00,4.000000,...,12.000000,31.000000,365.000000,6.000000,2018.000000,12.000000,31.000000,365.000000,6.000000,842 days 07:45:00


> date_diff_order_begin 可能要trim掉太大的值，奇怪的數值和跨年有關

In [22]:
# 把時間轉單位轉成總共幾分
df_predictors.go_time = df_predictors.go_time.apply(lambda x: x.total_seconds()/60) 
df_predictors.back_time = df_predictors.back_time.apply(lambda x: x.total_seconds()/60) 
df_predictors.date_diff_order_begin = df_predictors.date_diff_order_begin.apply(lambda x: x.total_seconds()/60) 

In [23]:
df_predictors.drop(['order_date_dt','begin_date_dt','end_date_dt'], axis=1)

,order_id,group_id,source_1,source_2,unit,people_amount,order_year,order_month,order_day,order_dayofyear,...,begin_month,begin_day,begin_dayofyear,begin_weekday,end_year,end_month,end_day,end_dayofyear,end_weekday,date_diff_order_begin
0,136100,63695,1,1,1,2,2017,1,5,5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,140370,63695,1,1,2,1,2016,11,9,314,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,211009,63695,1,1,3,2,2017,1,20,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,41571,53481,1,1,4,4,2017,3,14,73,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64125,53481,2,1,5,1,2017,3,27,86,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,76131,53481,2,2,6,2,2017,4,19,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,238671,53481,2,2,6,2,2017,4,19,109,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,5045,54305,1,1,7,6,2017,1,20,20,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,37230,54305,2,3,6,2,2017,5,8,128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,138976,54305,1,1,8,2,2017,5,8,128,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
airline_data[airline_data['group_id'] == '63695']

,group_id,go_back,fly_time,src_airport,arrive_time,dst_airport,src_IATA,dst_IATA,src_TZ,dst_TZ,arrive_time_utc,fly_time_utc,traval_time
0,63695,go,2017-03-22 23:30:00,TPE,2017-03-22 14:15:00,HNL,TPE,HNL,Asia/Taipei,Pacific/Honolulu,2017-03-23 00:15:00+00:00,2017-03-22 15:30:00+00:00,08:45:00
1,63695,back,2017-04-01 14:30:00,LAX,2017-04-02 21:00:00,TPE,LAX,TPE,America/Los_Angeles,Asia/Taipei,2017-04-02 13:00:00+00:00,2017-04-01 21:30:00+00:00,15:30:00


In [26]:
df_predictors[df_predictors['group_id'] == 63695]

,order_id,group_id,source_1,source_2,unit,people_amount,order_date_dt,order_year,order_month,order_day,...,begin_day,begin_dayofyear,begin_weekday,end_date_dt,end_year,end_month,end_day,end_dayofyear,end_weekday,date_diff_order_begin
0,136100,63695,1,1,1,2,2017-01-05,2017,1,5,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
1,140370,63695,1,1,2,1,2016-11-09,2016,11,9,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN
2,211009,63695,1,1,3,2,2017-01-20,2017,1,20,...,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
order_data[order_data['group_id']==63695]

,order_id,group_id,order_date,source_1,source_2,unit,people_amount,order_date_dt,order_year,order_month,order_day,order_dayofyear,order_weekday
0,136100,63695,5-Jan-17,1,1,1,2,2017-01-05,2017,1,5,5,3
1,140370,63695,9-Nov-16,1,1,2,1,2016-11-09,2016,11,9,314,2
2,211009,63695,20-Jan-17,1,1,3,2,2017-01-20,2017,1,20,20,4


In [28]:
df_total_travel_time[df_total_travel_time['group_id']=='63695']

,group_id,go_time,back_time
35785,63695,08:45:00,15:30:00


In [40]:
tmp = pd.merge(order_data[order_data['group_id']=='63695'], df_total_travel_time, on='group_id', how='left')
tmp

,order_id,group_id,order_date,source_1,source_2,unit,people_amount,order_date_dt,order_year,order_month,order_day,order_dayofyear,order_weekday,go_time,back_time
0,136100,63695,5-Jan-17,1,1,1,2,2017-01-05,2017,1,5,5,3,08:45:00,15:30:00
1,140370,63695,9-Nov-16,1,1,2,1,2016-11-09,2016,11,9,314,2,08:45:00,15:30:00
2,211009,63695,20-Jan-17,1,1,3,2,2017-01-20,2017,1,20,20,4,08:45:00,15:30:00


In [38]:
order_data[order_data['group_id']=='63695']

,order_id,group_id,order_date,source_1,source_2,unit,people_amount,order_date_dt,order_year,order_month,order_day,order_dayofyear,order_weekday
0,136100,63695,5-Jan-17,1,1,1,2,2017-01-05,2017,1,5,5,3
1,140370,63695,9-Nov-16,1,1,2,1,2016-11-09,2016,11,9,314,2
2,211009,63695,20-Jan-17,1,1,3,2,2017-01-20,2017,1,20,20,4


In [37]:
order_data['group_id'] = order_data['group_id'].apply(str)

In [34]:
df_total_travel_time['group_id'].head()

0    00018d44e062e3db9dc0460a5c78f0ee
1    000858b64a52e724cd008bbfe2fc8cdd
2    000a91291573c4b8b44300de8136b86b
3    000d1cccbb593a8210840a43457aa5e2
4    0012d75d4bfe483a414b09b50b548f58
Name: group_id, dtype: object

In [42]:
df_predictors.isna().sum(axis=0), df_predictors.shape

(order_id                      0
 group_id                      0
 source_1                      0
 source_2                      0
 unit                          0
 people_amount                 0
 order_date_dt                 0
 order_year                    0
 order_month                   0
 order_day                     0
 order_dayofyear               0
 order_weekday                 0
 go_time                  262408
 back_time                262392
 go_route_num             262408
 back_route_num           262392
 fly_time                 262408
 sub_line                 262144
 area                     262144
 days                     262144
 begin_date               262144
 price                    262144
 begin_date_dt            262144
 begin_year               262144
 begin_month              262144
 begin_day                262144
 begin_dayofyear          262144
 begin_weekday            262144
 end_date_dt              262144
 end_year                 262144
 end_month